In [1]:
import torch
torch.cuda.is_available()

True

In [7]:
import glob
import os
import numpy as np
import torch
import torch.optim as optim
import torchvision.transforms.functional as TF
from torchvision.transforms import v2
import torchvision.models as models
import torch.nn as nn
import cv2
from torch.utils.data import Dataset
from pathlib import Path
from typing import List, Tuple
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix


device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
cpu = torch.device('cpu')

Channels=4
IMG_SIZE=224
epochlist=[10]

Classes = ["higainashi", "houdo", "kanbotsu", "rokatahoukai"]
ClassNum = len(Classes)

testpath=r"C:\Users\AKIZUKI\JupyterProjects\QGISedited\train"
savepath=r"C:\Users\AKIZUKI\JupyterProjects\AerialPhoto_savedmodel"

'''
PytorchではDataloaderという,膨大なデータセットからでもメモリを圧迫せずに取り出せてforループにも対応するための枠組みがある
データセットをDataloaderが引っ張ってこれるような形式にするためにMyDataset(torch.utils.data.Dataset)というクラスを作れば，
あとはそのメソッドをtorch.utils.data.Datasetが勝手に使用してデータを加工してくれる
__init__, __getitem__, __len__をクラス内で必ず定義しなければならない
Dataloader内のデータはバッチごとにまとめられる
'''
class MyDataset(Dataset):
    def __init__(self, root: str, transforms, Classes) -> None:
        super().__init__()
        self.transforms = transforms
        self.Classes = Classes
        #globは複数のファイルのパスをまとめて取得する
        #訓練と訓練白黒の二個下のディレクトリから画像を取得
        self.data = list(sorted(Path(root).glob("*\*")))
        self.data2 = list(sorted(Path(root+"binary").glob("*\*")))



    # ここで取り出すデータを指定している
    def __getitem__(
            self,
            index: int
    ) -> Tuple[torch.Tensor, torch.Tensor]:

        data = self.data[index]
        data2 = self.data2[index]

        img1 = cv2.imread(str(data))
        img1 = cv2.resize(img1, (IMG_SIZE, IMG_SIZE))
        img2_tmp = cv2.imread(str(data2))

        # グレースケールに変換
        img2_tmp = cv2.cvtColor(img2_tmp,cv2.COLOR_BGR2GRAY)
        # 2値化
        ret,img2 = cv2.threshold(img2_tmp, 0, 255, cv2.THRESH_BINARY_INV)
        img2 = cv2.resize(img2, (IMG_SIZE, IMG_SIZE))

        #img1のテンソルとimg2のテンソルをチャンネル方向(dim0)に結合
        cat_img = torch.cat((TF.to_tensor(img1), TF.to_tensor(img2)), dim=0)

        # データの変形 (transforms)
        transformed_img = self.transforms(cat_img)

        #ラベル貼り：dataというパスを/で区切ってリストにし，クラス名のところをラベルに格納
        #クラス名は文字列なので，self.Classesの要素と比較して一致するところの番号をラベルとする
        label = str(data).split("\\")[-2]
        label = torch.tensor(self.Classes.index(label))

        return transformed_img, label

    # この method がないと DataLoader を呼び出す際にエラーを吐かれる
    def __len__(self) -> int:
        return len(self.data)


#入力データに施す処理
transforms = v2.Compose([
        v2.ToDtype(torch.float32, scale=True),
        v2.Normalize(mean=[0,0,0,0], std=[0.2, 0.2, 0.2, 0.2]),
])

testset= MyDataset(root=testpath, transforms=transforms, Classes=Classes)

testloader = DataLoader(dataset=testset,batch_size=len(testset),shuffle=True)
print(len(testloader))

resnet50 = models.resnet50()

#modify first layer so it expects 4 input channels; all other parameters unchanged
resnet50.conv1 = torch.nn.Conv2d(Channels,64,kernel_size = (7,7),stride = (2,2), padding = (3,3), bias = False)
#modifying final layer
resnet50.fc = nn.Linear(2048,ClassNum)

#lossfunction&optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet50.parameters(), lr=0.001, momentum=0.9)

def evaluate(testloader, model, loss_fn, optimizer):
    size_test = len(testloader.dataset)
    test_loss, test_correct = 0, 0
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    model.eval()
    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in testloader:
            X=X.to(device)
            y=y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            test_correct += (pred.argmax(1) == y).type(torch.float).sum().item()

        test_correct /= size_test

    print(f'TestLoss: {test_loss:.4f} TestAcc: {test_correct:.4f}')

    #ndarrayにするため、ラベルyと推測結果predをgpuからcpuへ返す
    y=y.to(cpu)
    pred=pred.to(cpu)
   
    y=np.array(y)

    #predは各クラスの確率になってる（onehotに近い）ので実際のクラス番号に戻す
    pred_class=pred.argmax(1)
    pred_class=np.array(pred_class)
    
    
    #テストデータの混同行列を計算し可視化
    #scikitlearnの混同行列はラベルをonehotではなく実際のクラス番号にする必要がある
    #混同行列の見方は行が正解ラベルのクラス列が推定クラス
    print(confusion_matrix(y, pred_class))
    print(" ")

for e in epochlist:
    #モデル構築
    modelpath = Path(savepath+"\\"+str(e)+"\model_weights.pth")
    epochmodel = resnet50
    epochmodel.load_state_dict(torch.load(modelpath))
    #GPUにニューラルネットワークを渡す
    epochmodel=epochmodel.to(device)

    print("Model in Epoch", e)
    #テストデータで評価
    evaluate(testloader, epochmodel, loss_fn, optimizer)

print('Testing Complete!!!')

1
Model in Epoch 10
TestLoss: 0.4563 TestAcc: 0.8659
torch.Size([328, 4])
(328,)
[[143  17   0   0]
 [  4 114   0   0]
 [  5   3   1   1]
 [  4  10   0  26]]
 
Testing Complete!!!
